In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, mixed_precision
from tensorflow.keras.utils import plot_model
from sklearn import preprocessing
import sys
from keras import backend as K
from sklearn.model_selection import train_test_split, StratifiedKFold
import matplotlib.pyplot as plt
import json
import gc
import sys, os, pickle, numpy as np, pandas as pd

2022-07-12 05:10:44.919184: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
config = {
    'data_path':'../../datasets/gtzan10sAug/datasets/AE1snr1BalAug/',
    'lr':1e-3,
    'epochs':10,
    'batch':256,
    'log_step':10,
    'multi_gpu':False,
    'gpu':2,
    'suffix':'denoising_AE',
}

In [11]:
import argparse
from operator import truediv
parser = argparse.ArgumentParser()
# Model configuration.
parser.add_argument('--data_path',type=str, default='../../datasets/gtzan10sAug/datasets/AE1snr1BalAug/', help='path to the folder containing train, test and val folders')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
parser.add_argument('--epochs', type=int, default=10, help='num epochs')
parser.add_argument('--multigpu', type=int, default=0, help='Train on multiple GPUs')
parser.add_argument('--gpu', type=str, default=2, help='GPU to use for training')
parser.add_argument('--log_step', type=int, default=10, help='log interval')
parser.add_argument('--batch', type=int, default = 256, help='Batch Size')
parser.add_argument('--suffix',type=str, default='denoising_AE', help='Model Name Suffix')
config = parser.parse_args([])

In [24]:
class AugmentedDataset(keras.utils.Sequence):
    def __init__(self, mode, batch_size):
        self.batch_size = batch_size
        self.mode = mode
        if self.mode == 'train': 
            self.noisy_path = os.path.join(config.data_path, 'features','train')
            self.clean_path = os.path.join(config.data_path, 'featuresClean','train')
        if self.mode == 'test':
            self.noisy_path = os.path.join(config.data_path, 'features','test')
            self.clean_path = os.path.join(config.data_path, 'featuresClean','test')
        if self.mode == 'val':
            self.noisy_path = os.path.join(config.data_path, 'features','val')
            self.clean_path = os.path.join(config.data_path, 'featuresClean','val')
        # print(os.path.exists(self.data_path))
        _,_,self.filenames = next(os.walk(self.noisy_path))

    def __len__(self):
        return int(len(self.filenames) // self.batch_size)

    def __getitem__(self,idx):
        try:
            source = np.empty((self.batch_size, 128,130))
            target = np.empty((self.batch_size, 128,130))
            batch = self.filenames[idx * self.batch_size : (idx+1) * self.batch_size]
            for i, ID in enumerate(batch):
                tmp = np.load(os.path.join(self.noisy_path, ID), allow_pickle=True)
                source[i] = tmp[0]
                tmp2 = np.load(os.path.join(self.clean_path, ID), allow_pickle=True)
                target[i] = tmp2[0]
            return source,target#X,y #bat[:,0], bat[:,1]
        except Exception as e:
            print(i, ID)
            print(e)


In [25]:
train = AugmentedDataset('train',256)

In [28]:
np.load(os.path.join(train.noisy_path, '949.npy'), allow_pickle=True)[0].shape

(128, 130)

In [27]:
np.load(os.path.join(train.clean_path, '949.npy'), allow_pickle=True)[0].shape

(128, 108)

In [26]:
next(iter(train))

8 949.npy
could not broadcast input array from shape (128,108) into shape (128,130)
